In [1]:
import pandas as pd
import requests

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

In [2]:
gkg = pd.read_csv("data/20240813.gkg.csv", sep="\t")
#gkg.head()

In [3]:
# create a function to apply to counts to split the string AFFECT#3000000#Americans#1#United States#US#US#39.828175#-98.5795#US; into a list. the separator is ";"

def split_counts(counts, sep=";"):
    if pd.isna(counts):
        return []
    return counts[:-1].split(sep)

# apply the function to the counts column

gkg["counts"] = gkg["COUNTS"].apply(split_counts)
gkg["themes"] = gkg["THEMES"].apply(split_counts)
gkg["locations"] = gkg["LOCATIONS"].apply(split_counts)
gkg["persons"] = gkg["PERSONS"].apply(split_counts)
gkg["organizations"] = gkg["ORGANIZATIONS"].apply(split_counts)
gkg["tone"] = gkg["TONE"].apply(split_counts, sep=",")
gkg["cameoeventids"] = gkg["CAMEOEVENTIDS"].apply(split_counts, sep=",")
gkg["sources"] = gkg["SOURCES"].apply(split_counts)
gkg["sourcesurl"] = gkg["SOURCEURLS"].apply(split_counts, sep='<UDIV>')


In [4]:
def get_dicts_from_lists(counts):
    if len(counts) == 0:
        return []
    return [dict(zip(["COUNTTYPE", "NUMBER", "OBJECTTYPE", "GEO_TYPE", "GEO_FULLNAME", "GEO_COUNTRYCODE", "GEO_ADM1CODE", "GEO_LAT", "GEO_LONG", "GEO_FEATUREID"], i.split("#"))) for i in counts]

gkg["counts"] = gkg["counts"].apply(get_dicts_from_lists)

In [5]:
# expand gkg["sourcesurl"] to create a new row for each value in the list

gkg.explode("sourcesurl")["sourcesurl"].head()

0    https://www.castanetkamloops.net/news/Business...
1    https://www.bostonglobe.com/2024/08/12/world/b...
2    https://www.wdrb.com/news/crime-reports/city-o...
3    https://www.brecorder.com/news/40317251/na-spe...
4    https://www.tribtoday.com/news/local-news/2024...
Name: sourcesurl, dtype: object

In [6]:
url = gkg.explode("sourcesurl")["sourcesurl"].iloc[0]

In [2]:
url = 'https://www.castanetkamloops.net/news/Business/501194/Starbucks-replaces-its-CEO-names-Chipotle-chief-to-head-the-compan'

In [3]:
from newspaper import Article

article = Article(url)

In [4]:
article.download()

In [5]:
article.parse()
article.nlp()

In [6]:
article.authors

['Dee-Ann Durbin', 'The Associated Press -']

In [7]:
article.text

'Photo: The Canadian Press FILE - A Starbucks mermaid logo sign is displayed on a store, May 29, 2024, in Salem, N.H. (AP Photo/Charles Krupa, File)\n\nStarbucks, struggling with weak demand and disgruntled investors, said Tuesday that CEO Laxman Narasimhan is stepping down after a little more than a year in the job.\n\nThe Seattle coffee giant said Brian Niccol, the chairman and CEO of Chipotle, will become Starbucks\' chairman and CEO on Sept. 9.\n\nStarbucks shares jumped more than 13% before the market opened.\n\nNarasimhan, a longtime PepsiCo executive who has also served as the CEO of Reckitt, a U.K.-based consumer health company, became Starbucks\' CEO in March 2023. He succeeded Howard Schultz, the longtime Starbucks leader and chairman emeritus who came out of retirement in 2022 to serve as the company\'s interim CEO.\n\nBut investors quickly soured on Narasimhan as the company\'s sales weakened and it dealt with multiple issues, including competition from lower-cost competito

In [9]:
help(article)

Help on Article in module newspaper.article object:

class Article(builtins.object)
 |  Article(url, title='', source_url='', config=None, **kwargs)
 |
 |  Article objects abstract an online news article page
 |
 |  Methods defined here:
 |
 |  __init__(self, url, title='', source_url='', config=None, **kwargs)
 |      The **kwargs argument may be filled with config values, which
 |      is added into the config object
 |
 |  build(self)
 |      Build a lone article from a URL independent of the source (newspaper).
 |      Don't normally call this method b/c it's good to multithread articles
 |      on a source (newspaper) level.
 |
 |  build_resource_path(self)
 |      Must be called after computing HTML/final URL
 |
 |  download(self, input_html=None, title=None, recursion_counter=0)
 |      Downloads the link's HTML content, don't use if you are batch async
 |      downloading articles
 |
 |      recursion_counter (currently 1) stops refreshes that are potentially
 |      infinite
 

In [10]:
article.__dict__

{'config': <newspaper.configuration.Configuration at 0x105dc2a80>,
 'extractor': <newspaper.extractors.ContentExtractor at 0x105dc11c0>,
 'source_url': 'https://www.castanetkamloops.net',
 'url': 'https://www.castanetkamloops.net/news/Business/501194/Starbucks-replaces-its-CEO-names-Chipotle-chief-to-head-the-compan',
 'title': 'Starbucks replaces its CEO, names Chipotle chief to head the company - Business News',
 'top_img': 'https://www.castanet.net/content/2024/8/37a61014e499717cd703e4a6c1730cc550663707ea732a0d131787b5c7e8e517_p3883387.jpg',
 'top_image': 'https://www.castanet.net/content/2024/8/37a61014e499717cd703e4a6c1730cc550663707ea732a0d131787b5c7e8e517_p3883387.jpg',
 'meta_img': 'https://www.castanet.net/content/2024/8/37a61014e499717cd703e4a6c1730cc550663707ea732a0d131787b5c7e8e517_p3883387.jpg',
 'imgs': {'https://c.castanet.net/img/45/5042861-1-1.jpg',
  'https://sb.scorecardresearch.com/p?c1=2&c2=6034663&cv=3.6.0&cj=1',
  'https://www.castanet.net/content/2024/8/37a61014